In [ ]:
import utils
import utils_stream
import utils_prob
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
def data_per_subj(eeg_folder, feats_path_folder, fsStim, bads, trial_len=60):
    eeg_files_all = [file for file in os.listdir(eeg_folder) if file.endswith('.set')]
    files = [file for file in eeg_files_all if len(file.split('_')) == 3]
    files.sort()
    eeg_list = []
    feats_list = []
    label_list = []
    for file in files:
        eeg_downsampled, eog_downsampled, fs = utils.get_eeg_eog(eeg_folder + file, fsStim, bads, expdim=False)
        eeg_reg = utils.regress_out(eeg_downsampled.T, eog_downsampled.T)
        len_video = eeg_reg.shape[0]
        name = file[:-4]
        id_att = name.split('_')[-1]
        ids = list(set(name.split('_')))
        f1 = utils.get_features(feats_path_folder, ids[0], len_video, offset=122*fs, smooth=True)
        f2 = utils.get_features(feats_path_folder, ids[1], len_video, offset=122*fs, smooth=True)
        feats = np.stack((f1[:,8], f2[:,8]), axis=1)
        label = 1 if id_att == ids[0] else 2

        # eeg_trials = utils.into_trials(eeg_reg, fs, t=trial_len)
        # feats_trials = utils.into_trials(feats, fs, t=trial_len)
        # label_trials = [label] * len(eeg_trials)

        # eeg_list += eeg_trials
        # feats_list += feats_trials
        # label_list += label_trials

        eeg_list.append(eeg_reg)
        feats_list.append(feats)
        label_list.append(label)

    return eeg_list, feats_list, label_list

In [ ]:
subjects = ['Pilot_1', 'Pilot_2', 'Pilot_4', 'Pilot_5', 'Pilot_6', 'Pilot_7', 'Pilot_8', 'Pilot_9', 'Pilot_10', 'Pilot_11', 'Pilot_12', 'Pilot_13', 'Pilot_14', 'Pilot_15', 'Pilot_17', 'Pilot_18', 'Pilot_19', 'Pilot_20', 'Pilot_21']
# bads = [['A30', 'B25'], ['B25'], ['B25'], [], ['A31', 'B31'], ['B25'], ['A30', 'B25'], ['A30', 'B25'], ['B25'], ['B25', 'B26'], ['A30', 'B25'], ['B31'], ['B25', 'A23'], ['A30', 'B25'], ['B25'], ['B25'], ['A30', 'B25'], ['A30', 'B25'], ['B25']] 
# eeg_dict = {}
# feats_dict = {}
# labels_dict = {}
# for subject, bad in zip(subjects, bads):
#     eeg_folder = f"../../Experiments/data/Two_Obj/Overlay/{subject}/"
#     feats_path_folder = '../Feat_Multi/features/'
#     fsStim = 30
#     eeg_trials, feats_trials, label_trials = data_per_subj(eeg_folder, feats_path_folder, fsStim, bad)
#     eeg_dict[subject] = eeg_trials
#     feats_dict[subject] = feats_trials
#     labels_dict[subject] = label_trials

# data_path = 'data/'
# if not os.path.exists(data_path):
#     os.makedirs(data_path)

# with open(data_path + 'eeg_video.pkl', 'wb') as f:
#     pickle.dump(eeg_dict, f)
# with open(data_path + 'feats_video.pkl', 'wb') as f:
#     pickle.dump(feats_dict, f)
# with open(data_path + 'labels_video.pkl', 'wb') as f:
#     pickle.dump(labels_dict, f)

In [ ]:
# load data
with open('data/eeg_video.pkl', 'rb') as f:
    eeg_dict = pickle.load(f)
with open('data/feats_video.pkl', 'rb') as f:
    feats_dict = pickle.load(f)
with open('data/labels_video.pkl', 'rb') as f:
    labels_dict = pickle.load(f)

In [ ]:
latent_dimensions = 5
fs = 30
hparadata = [3, 1]
hparafeats = [15, 0]
evalpara = [3, 2]
weightpara = [0.9, 0.9]
SEED = 100
PARATRANS = True
nb_calibsessions = 1
nb_disconnected = 0
sub_trial_length = 60
SHUFFLE = False
UPDATE_STEP = 60 // sub_trial_length if sub_trial_length else 1

In [ ]:
eeg_trials_dict = {}
feats_trials_dict = {}
labels_trials_dict = {}
for subject in eeg_dict.keys():
    eeg_videos = eeg_dict[subject]
    feats_videos = feats_dict[subject]
    labels_videos = labels_dict[subject]
    eeg_trials_all = []
    feats_trials_all = []
    labels_trials_all = []
    for eeg, feats, label in zip(eeg_videos, feats_videos, labels_videos):
        eeg_trials = utils.into_trials(eeg, fs, t=60)
        feats_trials = utils.into_trials(feats, fs, t=60)
        labels_trials = [label] * len(eeg_trials)
        eeg_trials_all += eeg_trials
        feats_trials_all += feats_trials
        labels_trials_all += labels_trials
    eeg_trials_dict[subject] = eeg_trials_all
    feats_trials_dict[subject] = feats_trials_all
    labels_trials_dict[subject] = labels_trials_all

In [ ]:
# Subj = 'Pilot_17'
# eeg_1 = eeg_trials_dict[Subj]
# feats_1 = feats_trials_dict[Subj]
# att_1, unatt_1 = utils_stream.select_att_unatt_feats(feats_1, labels_trials_dict[Subj])

# import algo_ccazoo
# views_list = [eeg_1, att_1]
# params_hankel = [(3, 1), (15, 0)]
# CCA = algo_ccazoo.CorrelationAnalysis(views_list, 'MCCA_LW', 30, params_hankel, CONTAIN_PARTIALS=False, n_components=3, leave_out=4, VALSET=False)
# corr_att, corr_unatt, corr_mm, model_fold = CCA.vad_mm(trial_len=60, feat_unatt_list=unatt_1, MM=True)

# utils.eval_compete(corr_att, corr_unatt, True, range_into_account=3, nb_comp_into_account=2)

In [ ]:
selected_subjects = ['Pilot_1', 'Pilot_5', 'Pilot_6', 'Pilot_10', 'Pilot_11', 'Pilot_12', 'Pilot_13', 'Pilot_14', 'Pilot_15', 'Pilot_17', 'Pilot_19', 'Pilot_20', 'Pilot_21']
# selected_subjects = subjects
est_subs = selected_subjects[:5]
est_corr_att_sum, est_corr_unatt_sum = utils_prob.estimate_distribution_corr(eeg_trials_dict, feats_trials_dict, labels_trials_dict, est_subs, fs, hparadata, hparafeats, leave_out_persubj=4, trial_len=60, range_into_account=evalpara[0], nb_comp_into_account=evalpara[1])
gmm_0, gmm_1 = utils_prob.fit_gmm(est_corr_att_sum, est_corr_unatt_sum, n_components_per_class=1)
selected_subjects = selected_subjects[5:]

In [ ]:
est_corr_att_unatt = np.stack([est_corr_att_sum, est_corr_unatt_sum], axis=1)
est_corr_unatt_att = np.stack([est_corr_unatt_sum, est_corr_att_sum], axis=1)
# 3. Create a grid to visualize the decision boundary
x_min = min(est_corr_unatt_att[:,0].min(), est_corr_att_unatt[:,0].min()) - 0.1
x_max = max(est_corr_unatt_att[:,0].max(), est_corr_att_unatt[:,0].max()) + 0.1
y_min = min(est_corr_unatt_att[:,1].min(), est_corr_att_unatt[:,1].min()) - 0.1
y_max = max(est_corr_unatt_att[:,1].max(), est_corr_att_unatt[:,1].max()) + 0.1

xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))
grid_points = np.c_[xx.ravel(), yy.ravel()]

# 4. Predict probabilities for grid points
grid_probs = utils_prob.predict_proba(grid_points, gmm_0, gmm_1)
blue_probs = grid_probs[:, 1].reshape(xx.shape)

# 5. Visualize the results
plt.figure(figsize=(10, 8))

# Plot the data points
plt.scatter(est_corr_unatt_att[:, 0], est_corr_unatt_att[:, 1], color='red', alpha=0.5, s=10, label='Class 0')
plt.scatter(est_corr_att_unatt[:, 0], est_corr_att_unatt[:, 1], color='blue', alpha=0.5, s=10, label='Class 1')

# Plot decision boundary (where probability = 0.5)
plt.contour(xx, yy, blue_probs, levels=[0.5], colors='black', linestyles='-', linewidths=2)

# Plot probability contours
contour = plt.contourf(xx, yy, blue_probs, levels=np.linspace(0, 1, 11), 
                      alpha=0.3)
plt.colorbar(contour, label='Probability of Class 1')


In [ ]:
data_subjects_dict = {}
feats_subjects_dict = {}
labels_subjects_dict = {}
rng = np.random.RandomState(SEED)
for subj in selected_subjects:
    data_trials = eeg_trials_dict[subj]
    if nb_disconnected > 0:
        disconnected_channels = rng.choice(data_trials[0].shape[1], nb_disconnected, replace=False)
        for i in range(len(data_trials)):
            data_trials[i][:, disconnected_channels] = 0
    data_trials = [utils_stream.process_data_per_view(d, hparadata[0], hparadata[1], NORMALIZE=True) for d in data_trials]
    feats_trials = feats_trials_dict[subj]
    feats_trials = [utils_stream.process_data_per_view(f, hparafeats[0], hparafeats[1], NORMALIZE=True) for f in feats_trials]
    labels_trials = labels_trials_dict[subj]
    if sub_trial_length is not None:
        data_trials, feats_trials, labels_trials = utils_stream.further_split_and_shuffle(data_trials, feats_trials, labels_trials, sub_trial_length, fs, SHUFFLE=SHUFFLE, SEED=SEED)
    data_subjects_dict[subj] = data_trials
    feats_subjects_dict[subj] = feats_trials
    labels_subjects_dict[subj] = labels_trials
nb_trials = 32

In [ ]:
stream = utils_stream.STREAM(data_subjects_dict, feats_subjects_dict, hparadata[0], hparafeats[0], latent_dimensions, SEED, evalpara, nb_trials, UPDATE_STEP)
true_labels = np.concatenate([v[:nb_trials*UPDATE_STEP] for v in labels_subjects_dict.values()])

pred_labels_dict = stream.fixed_supervised(labels_subjects_dict, selected_subjects[:nb_calibsessions], selected_subjects[nb_calibsessions:])
pred_labels_fixed = np.concatenate([v for v in pred_labels_dict.values()])

pred_labels_dict = stream.adaptive_supervised(labels_subjects_dict, weightpara, PARATRANS=PARATRANS)
pred_labels_adapsup = np.concatenate([v for v in pred_labels_dict.values()])

pred_labels_dict = stream.recursive(weightpara, PARATRANS=PARATRANS, SINGLEENC=True)
pred_labels_single = np.concatenate([v for v in pred_labels_dict.values()])

pred_labels_dict = stream.recursive(weightpara, PARATRANS=PARATRANS, SINGLEENC=False)
pred_labels_two = np.concatenate([v for v in pred_labels_dict.values()])

pred_labels_dict = stream.recursive_soft(weightpara, gmm_0, gmm_1, PARATRANS=PARATRANS)
pred_labels_soft = np.concatenate([v for v in pred_labels_dict.values()])

In [ ]:
print("###########Fixed Supervised###########")
acc_non_calib_fixed, acc_fixed = utils_stream.calc_smooth_acc(pred_labels_fixed, true_labels, nb_trials, UPDATE_STEP, nearby=14, nb_calibsessions=nb_calibsessions)
print("###########Adaptive Supervised###########")
acc_non_calib_adapsup, acc_adapsup = utils_stream.calc_smooth_acc(pred_labels_adapsup, true_labels, nb_trials, UPDATE_STEP, nearby=14, nb_calibsessions=nb_calibsessions)
print("###########Single-Encoder###########")
acc_non_calib_single_enc, acc_single = utils_stream.calc_smooth_acc(pred_labels_single, true_labels, nb_trials, UPDATE_STEP, nearby=14, nb_calibsessions=nb_calibsessions)
print("###########Two-Encoder###########")
acc_non_calib, acc_two = utils_stream.calc_smooth_acc(pred_labels_two, true_labels, nb_trials, UPDATE_STEP, nearby=14, nb_calibsessions=nb_calibsessions)
print("###########Soft-Recursive###########")
acc_non_calib_soft, acc_soft = utils_stream.calc_smooth_acc(pred_labels_soft, true_labels, nb_trials, UPDATE_STEP, nearby=14, nb_calibsessions=nb_calibsessions)

In [ ]:
plt.close('all')
plt.figure(figsize=(10, 3))
x_axis = np.arange(len(true_labels))/UPDATE_STEP
plt.plot(x_axis, acc_two, label='Two-Enc', color='orange')
plt.plot(x_axis, acc_single, label='Single-Enc', color='blue')
plt.plot(x_axis, acc_adapsup, label='Adaptive Supervised', color='red')
plt.plot(x_axis[nb_calibsessions*nb_trials*UPDATE_STEP:], acc_fixed, label='Fixed Supervised', color='green')
plt.plot(x_axis, acc_soft, label='Soft-Recursive', color='purple')
# mark recur_acc_se and recur_acc_te in title
plt.title(f"FixSup: {acc_non_calib_fixed:.2f}, Single-Enc: {acc_non_calib_single_enc:.2f}, Two-Enc: {acc_non_calib:.2f}, Soft: {acc_non_calib_soft:.2f}, AdaSup: {acc_non_calib_adapsup:.2f}")
plt.xlabel('time (min)')
plt.ylabel('accuracy')
for i in range(nb_calibsessions, len(selected_subjects)):
    plt.axvline(x=i*nb_trials, color='grey', linestyle='--')
plt.legend()
plt.show()